In [1]:
import dask.array as da
import numpy as np

DATASET_PATH = "/data/smartspim_zarr_test/Ex_488_Em_525/429940_372540.zarr/0"
lazy_data = da.from_zarr(DATASET_PATH)[0, 0, ...]
lazy_data

dask.array<getitem, shape=(4308, 1600, 2000), dtype=uint16, chunksize=(128, 128, 128), chunktype=numpy.ndarray>

In [2]:
from pathlib import Path

def pick_slices(image_stack, percentage, read_lazy=True):
    """
    Pick slices from a 3D image stack based on a given percentage.
    
    Args:
    - image_stack: 3D numpy array representing the image stack (Z, Y, X).
    - percentage: Percentage of the Z stack to pick (between 0 and 1).
    
    Returns:
    - picked_slices: List of slices picked from the image stack.
    """
    z_dim = image_stack.shape[0]
    num_slices_to_pick = int(np.floor(percentage * z_dim))
    
    if num_slices_to_pick == 0:
        raise ValueError("Percentage too low to pick any slices.")
    
    step_size = z_dim // num_slices_to_pick
    
    slices = list(range(0, z_dim, step_size))
    
    picked_slices = None
    if read_lazy:
        picked_slices = [image_stack[i] for i in slices]
    
    return picked_slices, slices

data_folder = Path("/data/smartspim_zarr_test/Ex_488_Em_525") 

In [9]:
def get_slices(zarr_path, indices):
    worker_results = {}
    
    for index in indices:
        print(f"Worker {os.getpid()} reading slide {index} from {zarr_path}\n")
        worker_results[zarr_path.stem] = da.from_zarr(zarr_path)[0, 0, index, ...].compute()
    
    return worker_results

def _execute_worker(params):
    """
    Worker interface to provide parameters
    """
    return get_slices(**params)

import multiprocessing
import os

exec_n_workers = 16

# Create a pool of processes
pool = multiprocessing.Pool(processes=exec_n_workers)
final_results = []

picked_slices, indices = pick_slices(lazy_data, percentage=0.005, read_lazy=False)
scale = 0
folders = list(data_folder.glob("*.zarr"))

print(indices, len(indices))

picked_blocks = []
curr_picked_blocks = 0

for zarr_path in folders:
    print(zarr_path)
    params = {
        "zarr_path": Path(zarr_path).joinpath(str(scale)),
        "indices": indices,
    }
    picked_blocks.append(params)
    
    curr_picked_blocks += 1
    
    if curr_picked_blocks == exec_n_workers:
        # Assigning blocks to execution workers
        jobs = [
            pool.apply_async(_execute_worker, args=(picked_block,))
            for picked_block in picked_blocks
        ]

        print(
            f"Dispatcher PID {os.getpid()} dispatching {len(jobs)}"
        )

        # Wait for all processes to finish
        results = [job.get() for job in jobs]  # noqa: F841
        final_results.append(results)
        
        # Setting variables back to init
        curr_picked_blocks = 0
        picked_blocks = []

if curr_picked_blocks != 0:
    print(f"Blocks not processed inside of loop: {curr_picked_blocks}")
    # Assigning blocks to execution workers
    jobs = [
        pool.apply_async(_execute_worker, args=(picked_block,))
        for picked_block in picked_blocks
    ]

    # Wait for all processes to finish
    results = [job.get() for job in jobs]  # noqa: F841
    final_results.append(results)

    # Setting variables back to init
    curr_picked_blocks = 0
    picked_blocks = []

# Closing pool of workers
pool.close()

Worker 21714 reading slide 0 from /data/smartspim_zarr_test/Ex_488_Em_525/429940_320700.zarr/0
Worker 21712 reading slide 0 from /data/smartspim_zarr_test/Ex_488_Em_525/429940_268860.zarr/0
Worker 21716 reading slide 0 from /data/smartspim_zarr_test/Ex_488_Em_525/429940_372540.zarr/0
Worker 21713 reading slide 0 from /data/smartspim_zarr_test/Ex_488_Em_525/429940_294780.zarr/0
Worker 21718 reading slide 0 from /data/smartspim_zarr_test/Ex_488_Em_525/429940_424380.zarr/0
Worker 21717 reading slide 0 from /data/smartspim_zarr_test/Ex_488_Em_525/429940_398460.zarr/0
Worker 21715 reading slide 0 from /data/smartspim_zarr_test/Ex_488_Em_525/429940_346620.zarr/0
Worker 21719 reading slide 0 from /data/smartspim_zarr_test/Ex_488_Em_525/462340_268860.zarr/0
Worker 21720 reading slide 0 from /data/smartspim_zarr_test/Ex_488_Em_525/462340_294780.zarr/0
Worker 21721 reading slide 0 from /data/smartspim_zarr_test/Ex_488_Em_525/462340_320700.zarr/0
Worker 21722 reading slide 0 from /data/smartspim_

Process ForkPoolWorker-71:
Process ForkPoolWorker-78:
Process ForkPoolWorker-79:
Process ForkPoolWorker-65:


KeyboardInterrupt: 

Process ForkPoolWorker-68:
Process ForkPoolWorker-72:
Process ForkPoolWorker-75:
Process ForkPoolWorker-66:
Process ForkPoolWorker-74:
Process ForkPoolWorker-77:
Process ForkPoolWorker-70:
Process ForkPoolWorker-80:
Process ForkPoolWorker-76:
Process ForkPoolWorker-73:
Process ForkPoolWorker-67:
Process ForkPoolWorker-69:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/envs/flatfield_estimation/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/envs/flatfield_estimation/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/envs/flatfield_est